## 1 Problem Statement


Online Retail, a company specializing in e-commerce, recently invested a substantial portion of its revenue in an advertising campaign to boost brand and product awareness. Despite these efforts, the campaign achieved an acquisition response rate of only 3%, falling short of the anticipated 6%. Management suspects that the campaign's underperformance stemmed from its broad and costly approach, which failed to consider the diverse purchasing behaviors of customers.

To improve outcomes, the company intends to focus future marketing efforts on customers most likely to drive revenue growth. With the next campaign scheduled in six months, management seeks to achieve the following objectives:

- Customer Value Analysis: Assess the commercial value of each customer just before the campaign launch.
- Customer Segmentation: Develop a segmentation strategy based on purchasing behaviors to identify key customer groups.
- Marketing Enablement Tool: Equip the Marketing team with a tool to implement and sustain a targeted marketing strategy.
- 
The Data Science team has been tasked with leading this project. They will collaborate with the Marketing team responsible for promotions, the Technology team, and a Management Committee representative. Although the company's database contains some data gaps due to past system migrations, it will serve as the foundation for this initiative.
The success of this project will be evaluated based on the achievement of the targeted response rate of 6%, a key performance metric set by management.


## 2.0 Import the common libraries

In [1]:
# Code task 1#
# Import the library needed for this project
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt

## 3.0 Import the Data

In [2]:
# Code task 2#
# Load the raw data
df = pd.read_csv('Online_Retail_Data_Set.csv', encoding='utf-8', encoding_errors='ignore')

In [3]:
# Code task 3#
# Call the info method to see a summary of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
# Code task 4#
# Call on the describe method to see some statistics related to the data
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
# Code task 5#
# Call the head method
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 8:26,3.39,17850.0,United Kingdom


## 4.0 Exploring the Data

### 4.1 Identifying Missing Values and Cleaning

#### 4.1.1 Number of missing values by column

In [6]:
# Code task 5#
#Count the number of missing value in each column using 'sum()' as well as the percentages using 'mean()'
#Order them using sort_values
#Call pd.concat to present these in a single DataFrame
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns = ['count', '%']
missing.sort_values(by='%', ascending=False)


,count,%
CustomerID,135080,24.926694
Description,1454,0.268311
InvoiceNo,0,0.000000
StockCode,0,0.000000
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
Country,0,0.000000


### 4.2 Format Columns Appropriately

#### 4.2.1 Convert the column 'InvoiceDate' in a datetime format

In [7]:
# Code task 6#
#Convert InvoiceDate in a datetime format
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'], dayfirst=True)

In [8]:
# Code task 7#
#Check on the format shift of the InvoiceDate
print(df.dtypes)

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


#### 4.2.2 Convert the column 'Country' into categorical format

In [9]:
# Code task 8#
# Convert the column 'Country' into categorical format
df['Country'] = df['Country'].astype('category')
print(df.dtypes)

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country              category
dtype: object


In [10]:
# Code task 9# 
# Number of Countries Where the company sells its products
print(df['Country'].nunique())

# Code task 10#
# Countries Where the company Sells its Products
for country in df['Country'].unique():
     print(country)

38
United Kingdom
France
Australia
Netherlands
Germany
Norway
EIRE
Switzerland
Spain
Poland
Portugal
Italy
Belgium
Lithuania
Japan
Iceland
Channel Islands
Denmark
Cyprus
Sweden
Austria
Israel
Finland
Bahrain
Greece
Hong Kong
Singapore
Lebanon
United Arab Emirates
Saudi Arabia
Czech Republic
Canada
Unspecified
Brazil
USA
European Community
Malta
RSA


#### 4.2.3 Column InvoiceNo
We choose to keep the InvoiceNo column as an object data type since we do not intend to perform any calculations with it.

### 4.3 Clarify Column Labels and Their Contents

#### 4.3.1 Label European Country

France, the Netherlands, Germany, Spain, Poland, Portugal, Italy, Belgium, Lithuania, Denmark, Cyprus, Sweden, Austria, Finland, Greece, the Czech Republic, and Malta are members of the European Community. Including 'European Community' in the list of countries alongside these nations is therefore inappropriate. Let's estimate the number of customers affected by this condition.

In [11]:
# Code task 11#
#Extract Customers Located in the European Community from the Dataset
df_European_Community = df[df['Country']=='European Community']

# Code task 12# 
# Count the number of unique customers from the European Community
df_European_Community.CustomerID.nunique()

1

In [12]:
# Code task 13#
# CustomerID of the customer from the European Community
df_European_Community['CustomerID'].unique()

array([15108.])

###### There is only one customer. We will replace the label in the 'European Community' column with 'Other European Country'.

#### 4.3.2 Standardization of labels

To maintain uniformity in the Country column, we will replace the following abbreviations:

USA = United States

RSA = Republic of South Africa

In [13]:
# Code task 14#
# Change the labels of some contents in 'Country' column
df['Country'] = df['Country'].replace ({'European Community' : 'Other European Country',\
                                        'USA' : 'United States', \
                                        'RSA': 'Republic of South Africa'})
# Code task 15#
#Check on the modification
df.Country.unique()

['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany', ..., 'Brazil', 'United States', 'Other European Country', 'Malta', 'Republic of South Africa']
Length: 38
Categories (38, object): ['Australia', 'Austria', 'Bahrain', 'Belgium', ..., 'United States', 'United Arab Emirates', 'United Kingdom', 'Unspecified']

### 4.4 Adding some columns to future analyse

#### 4.3.1 Decompose the InvoiceDate column into Year, Month, and Day to facilitate analysis across different time scales.

In [14]:
# Code task 9#
# Insert columns 'Year', 'Month','Day' in the DataFrane
df['Month'] =df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year
df['Day'] = df['InvoiceDate'].dt.day

# Code task 10#
# Check on the changes
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Month', 'Year', 'Day'],
      dtype='object')

#### 4.3.2 Insert a new column : Revenue

The primary indicator of the company's performance is its profitability. Although this metric is not present in the original dataset, it can be calculated using the formula: Revenue = Quantity × Unit Price.

In [15]:
# Code task 10#
#Calculate Revenue for each customer 
df['Revenue'] = df['Quantity']*df['UnitPrice']

# Code task 11#
# Check on the change
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Month', 'Year', 'Day',
       'Revenue'],
      dtype='object')

#### 4.3.3 Insert a column 'Continent'

In [16]:
# Code task 12#
# Define a dictionary mapping countries to continents
country_to_continent = {
    'United Kingdom': 'Europe',
    'France': 'Europe',
    'Australia': 'Oceania',
    'Netherlands': 'Europe',
    'Germany': 'Europe',
    'Norway': 'Europe',
    'EIRE': 'Europe',
    'Switzerland': 'Europe',
    'Spain': 'Europe',
    'Poland': 'Europe',
    'Portugal': 'Europe',
    'Italy': 'Europe',
    'Belgium': 'Europe',
    'Lithuania': 'Europe',
    'Japan': 'Asia',
    'Iceland': 'Europe',
    'Channel Islands': 'Europe',
    'Denmark': 'Europe',
    'Cyprus': 'Europe',
    'Sweden': 'Europe',
    'Austria': 'Europe',
    'Israel': 'Asia',
    'Finland': 'Europe',
    'Bahrain': 'Asia',
    'Greece': 'Europe',
    'Hong Kong': 'Asia',
    'Singapore': 'Asia',
    'Lebanon': 'Asia',
    'United Arab Emirates': 'Asia',
    'Saudi Arabia': 'Asia',
    'Czech Republic': 'Europe',
    'Canada': 'North America',
    'Unspecified': 'Unspecified',
    'Brazil': 'South America',
    'United States': 'North America',
    'Other European Country': 'Europe',
    'Malta': 'Europe',
    'Republic of South Africa': 'Africa'
}

# Code task 13#
# Add a new column 'Continent' to the DataFrame
df['Continent'] = df['Country'].map(country_to_continent)

# Code task 
# Verify the changes
print(df[['Country','Continent']])


               Country Continent
0       United Kingdom    Europe
1       United Kingdom    Europe
2       United Kingdom    Europe
3       United Kingdom    Europe
4       United Kingdom    Europe
...                ...       ...
541904          France    Europe
541905          France    Europe
541906          France    Europe
541907          France    Europe
541908          France    Europe

[541909 rows x 2 columns]


### 4.4 Remove rows without CustomerID

In [17]:
df = df.dropna(subset=['CustomerID'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  category      
 8   Month        406829 non-null  int32         
 9   Year         406829 non-null  int32         
 10  Day          406829 non-null  int32         
 11  Revenue      406829 non-null  float64       
 12  Continent    406829 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(3), int32(3), int64(1), object(4)
memory usage: 36.1+ MB


## 5.0 Save the Clean Data

In [18]:
df.to_csv('C:/Users/wgerm/OneDrive/Documents/GitHub/Capstone 3/Online_Retail_Dataset/df_clean.csv', index=False)

## 6.0 Summary

The Online_retail dataset imported from Kaggle underwent the following modifications to prepare it for analysis:

- 135,079 rows without a unique customer identifier were removed, reducing the dataset from 541,908 rows to 406,829 rows.

- The data types of the following columns were modified:

    . InvoiceDate: from object to datetime64[ns]

    . Country: from object to category

- Five new columns were added: Month, Year, Day, Revenue, and Continent.

- Corrections were made to the labels of cities and the Country column.

As a result, the dataset dimensions changed from 541,908 x 7 to a clean dataset of 406,829 x 12, ready for analysis.

## FIN